# MATH 307  --   What is it useful for

## Interpolation of data points

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

Lets imaging you what to interpolate between some given data points for a project with some scientist on the other side of the earth. 

You get the following data point
(x_0, y_0) = (-1, 8) ; (x_1,y_1) = (-0.8, 5.5) ; (x_2,y_2) = (0, 3)

In [ ]:
x = np.array([-1, -0.8, 0])
y = np.array([8, 5.5, 3])
A = np.vander(x,increasing=True)

In [ ]:
print(A)

The matrix A gives use a linear system we can solve to find the interpolation between these points 

In [ ]:
c = la.solve(A,y)

In [ ]:
print(c)

This is the solution of the system Ac = y. Lets see how the plot looks like

In [ ]:
T = np.linspace(-1,0,100)
Y = c[0] + c[1]*T + c[2]*T**2 
plt.plot(T,Y,'b-',x,y,'r.',markersize=10)
plt.grid(True)
plt.show()

Then an email reaches you and you get the information that there has to be the additional constraint Y(-1/3) = 3. 
In other words you get an additional data point (x_3, y_3) = (-1/3, 3) 

In [ ]:
x = np.array([-1, -0.8, -1/3, 0])
y = np.array([8, 5.5,3,3])
A = np.vander(x,increasing=True)

In [ ]:
print(A)

In [ ]:
c = la.solve(A,y)

In [ ]:
print(c)

In [ ]:
T = np.linspace(-1,0,100)
Y = c[0] + c[1]*T + c[2]*T**2 + c[3]*T**3 
plt.plot(T,Y,'b-',x,y,'r.',markersize=10)
plt.grid(True)
plt.show()

That is still all fine. Now lets assume we get the following set of data points which was constructed by a lot of measurements with a very faulty measurement device.

In [ ]:
c0 = 3
c1 = 5
c2 = 8
c3 = -2
N = 1000
t = np.random.rand(N) - 1 # Random numbers in the interval (-1,1)
noise = np.random.randn(N)
y = c0 + c1*t + c2*t**2 + c3*t**3 + noise
plt.scatter(t,y,alpha=0.5,lw=0,s=10);
plt.show()

The plot of the datapoints has resemblence with the previous plot. But how do we find a solution to describe these data points with minimal error.

In [ ]:
A = np.column_stack([np.ones(N),t,t**2])

We solve $\left( A^T A \right) \mathbf{c} = \left( A^T \right) \mathbf{y}$ which gives us the least-square solution

In [ ]:
c = la.solve(A.T @ A,A.T @ y)


In [ ]:
ts = np.linspace(-1,0,20)
ys = c[0] + c[1]*ts + c[2]*ts**2 
plt.plot(ts,ys,'r',linewidth=4)
plt.scatter(t,y,alpha=0.5,lw=0)
plt.show()

### Image deblurring

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
kitten = plt.imread('data/kitten.jpg').astype(np.float64)

In [ ]:
N = 256
c = np.zeros(N)
s = 5
c[:s] = (s - np.arange(0,s))/(3*s)
Ac = la.toeplitz(c)
r = np.zeros(N)
s = 20
r[:s] = (s - np.arange(0,s))/(3*s)
Ar = la.toeplitz(r)

In [ ]:
B = Ac@kitten@Ar.T + 0.01*np.random.randn(256,256)

In [ ]:
plt.imshow(B,cmap='gray')
plt.show()

The image of the kitten is blurred by some noise $E$, so that 

$$
A_c X A_r^T = B + E
$$

How do we find $X$, i.e. the unblurred image of the kitten?

We compute using the truncated pseudoinverse

$$
X = (A_c)_k^+ B (A_r^T)_k^+
$$

In [ ]:
Pc,Sc,QTc = la.svd(Ac)
Pr,Sr,QTr = la.svd(Ar)

In [ ]:
k = 50
Dc_k_plus = np.hstack([1/Sc[:k],np.zeros(N-k)])
Dr_k_plus = np.hstack([1/Sr[:k],np.zeros(N-k)])
Ac_k_plus = QTc.T @ np.diag(Dc_k_plus) @ Pc.T
Ar_k_plus = Pr @ np.diag(Dr_k_plus) @ QTr
X = Ac_k_plus @ B @ Ar_k_plus

In [ ]:
plt.imshow(X,cmap='gray')
plt.show()

In [ ]:
k = 100
Dc_k_plus = np.hstack([1/Sc[:k],np.zeros(N-k)])
Dr_k_plus = np.hstack([1/Sr[:k],np.zeros(N-k)])
Ac_k_plus = QTc.T @ np.diag(Dc_k_plus) @ Pc.T
Ar_k_plus = Pr @ np.diag(Dr_k_plus) @ QTr
X = Ac_k_plus @ B @ Ar_k_plus

In [ ]:
plt.imshow(X,cmap='gray')
plt.show()

In [ ]:
k = 200
Dc_k_plus = np.hstack([1/Sc[:k],np.zeros(N-k)])
Dr_k_plus = np.hstack([1/Sr[:k],np.zeros(N-k)])
Ac_k_plus = QTc.T @ np.diag(Dc_k_plus) @ Pc.T
Ar_k_plus = Pr @ np.diag(Dr_k_plus) @ QTr
X = Ac_k_plus @ B @ Ar_k_plus

In [ ]:
plt.imshow(X,cmap='gray')
plt.show()

In [ ]:
plt.imshow(kitten,cmap='gray')
plt.show()

**Homework Exercises**

1. Find the general solution of the linear system:  $$
\begin{array}{ r @{{}={}} r  >{{}}c<{{}} r  >{{}}c<{{}}  r }
x_1 &-& 3x_2 &+& 4x_3 &=&  1 \\
-2x_1 &+&  5x_2 &-& 7x_3 &=& 1 \\
x_1 &-& 5x_2 &+& 8x_3 &=& 5 \\
\end{array}
$$


2. Does the matrix equation $Ax = b$ have a solution? If yes, write down the general solution of the system:  
$$ 
A = 
\begin{bmatrix}
2 & -8 & 4 & 2 \\
1 & -3 & 0 & 2 \\
-1 & 2 & 2 & -4 \\
-3 & 11 & -4 & 2
\end{bmatrix}\,, \quad 
b = \begin{bmatrix}
1 \\ 2 \\ 5 \\ 2
\end{bmatrix}
$$

3. What about the equation $Bx = c$? If it has a solution, write it down in the most general form:    
$$
B = \begin{bmatrix}
1 & 1 & 1 & 1 \\
1 & 1 & -1 & -1 \\
1 & -1 & 0 & 0 \\
0 & 0 & 1 & 1
\end{bmatrix}\,, \quad 
c = \begin{bmatrix}
3 \\ 1 \\ 0 \\ 1
\end{bmatrix}
$$